# Getting all Event Data from The Graph
Below is the process used to pull all network events from The Graph. 

In [1]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

In [2]:
# function used to query each type of event from The Graph
def query_theGraph(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}"
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + indexer + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [3]:
#get allocationCreated event
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

created_query = '''query{
  allocationCreateds() {
    id
    blockNumber
    logIndex
    indexer
    subgraphDeploymentID
    epoch
    tokens
    allocationID
  }  
}
'''

field_name = 'allocationCreateds'

alloc_created_df = query_theGraph(created_query, field_name, url, False)

In [4]:
#save allocationCreated data
alloc_created_df['type'] = 'allocationCreateds'
alloc_created_df.to_csv('allocationCreateds.csv', index = False, header = True)

In [5]:
# get allocationClosed event
close_query = """query{
  allocationCloseds() {
    id
    blockNumber
    logIndex
    indexer
    subgraphDeploymentID
    epoch
    tokens
    allocationID
    sender
    isDelegator
  }
  
}"""

field_name = 'allocationCloseds'

alloc_close_df = query_theGraph(close_query, field_name, url, False)

#save allocationClosed data
alloc_close_df['type'] = 'allocationCloseds'
alloc_close_df.to_csv('allocationCloseds.csv', index = False, header = True)

In [6]:
# get allocationCollected event
collect_query = """query{
  allocationCollecteds() {
    id
    blockNumber
    logIndex
    indexer
    subgraphDeploymentID
    epoch
    tokens
    allocationID
    from
    curationFees
    rebateFees
  }
  
}"""

field_name = 'allocationCollecteds'

alloc_collected_df = query_theGraph(collect_query, field_name, url, False)

# save allocationCollected data
alloc_collected_df['type'] = 'allocationCollecteds'
alloc_collected_df.to_csv('allocationCollected.csv', index = False, header = True)

In [7]:
# get stakeDelegated event
delegate_query = """query{
  stakeDelegateds () {
    id
    blockNumber
    logIndex
    indexer
    delegator
    tokens
    shares
  }
}"""

field_name = 'stakeDelegateds'

stake_delegate_df = query_theGraph(delegate_query, field_name, url, False)

# save stakeDelegated data
stake_delegate_df['type'] = 'stakeDelegateds'
stake_delegate_df.to_csv('stakeDelegateds.csv', index = False, header = True)

In [8]:
# get stakeLocked event
locked_query = """query{
  stakeDelegatedLockeds() {
    id
    blockNumber
    logIndex
    eventTime
    indexer
    delegator
    tokens
    shares
    until
  }
 }"""

field_name = 'stakeDelegatedLockeds'

stake_locked_df = query_theGraph(locked_query, field_name, url, False)

# save stakeLocked data
stake_locked_df['type'] = 'stakeDelegatedLockeds'
stake_locked_df.to_csv('stakeDelegatedLockeds.csv', index = False, header = True)

In [9]:
#get stakeWithdraw event
withdraw_query = """query{stakeDelegatedWithdrawns () {
    id
    blockNumber
    logIndex
    indexer
    delegator
    tokens
}
}"""

field_name = 'stakeDelegatedWithdrawns'

stake_withdraw_df = query_theGraph(withdraw_query, field_name, url, False)

# save stakeWithdraw data
stake_withdraw_df['type'] = 'stakeDelegatedWithdrawns'
stake_withdraw_df.to_csv('stakeDelegatedWithdrawns.csv', index = False, header = True)

In [10]:
# get fee cut data
fee_query = """query{delegationParametersUpdateds () {
    id
    logIndex
    blockNumber
    indexer
    indexingRewardCut
    queryFeeCut
}
}"""
field_name = 'delegationParametersUpdateds'
fee_cut_df = query_theGraph(fee_query, field_name, url, False)

# save fee cut data
fee_cut_df['type'] = 'delegationParametersUpdateds'
fee_cut_df.to_csv('delegationParametersUpdateds.csv', index = False, header = True)

In [11]:
# get rebateClaim data
rebate_query = """query{rebateClaimeds () {
    id
    blockNumber
    logIndex
    indexer
    tokens
    subgraphDeploymentID
    allocationID
    delegationFees
    epoch
}
}"""
field_name = 'rebateClaimeds'
rebate_claim_df = query_theGraph(rebate_query, field_name, url, False)

# save rebateClaim data
rebate_claim_df['type'] = 'rebateClaimeds'
rebate_claim_df.to_csv('rebateClaimeds.csv', index = False, header = True)

In [12]:
# get rewardsAssigned data
rewardsassigned_query = """query{rewardsAssigneds () {
    id
    blockNumber
    logIndex
    indexer
    amount
    epoch
}
}"""
field_name = 'rewardsAssigneds'
rewards_assigned_df = query_theGraph(rewardsassigned_query, field_name, url, False)

# save rewardsAssigned data
rewards_assigned_df['type'] = 'rewardsAssigneds'
rewards_assigned_df.to_csv('rewardsAssigneds.csv', index = False, header = True)

In [13]:
# get stakeDeposited data
index_delegate_query = """query{stakeDepositeds () {
    id
    blockNumber
    logIndex
    indexer
    tokens
}
}"""
field_name = 'stakeDepositeds'
stake_deposit_df = query_theGraph(index_delegate_query, field_name, url, False)

# save stakeDeposited data
stake_deposit_df['type'] = 'stakeDepositeds'
stake_deposit_df.to_csv('stakeDepositeds.csv', index = False, header = True)

In [15]:
# combine all events
combined_events = [alloc_created_df, alloc_close_df, alloc_collected_df, 
                   stake_delegate_df, stake_locked_df, stake_withdraw_df, 
                   fee_cut_df, rebate_claim_df, rewards_assigned_df, stake_deposit_df]
combined_events = pd.concat(combined_events)
combined_events.drop_duplicates(inplace = True)
combined_events.to_csv('../multiple_indexer/allindexer/allEvents.csv', index = False, header=True)